In [1]:
# coding=utf-8
"""
Auction simulation for a Double Auction game.

Implements:
  - Buyer and Seller classes with placeholder strategies.
  - Auction class managing discrete time steps and sub-steps.
  - Logging to Pandas DataFrames for all steps and trades.
"""

import random
import pandas as pd
from typing import List, Optional, Tuple

class Buyer:
  """Represents a buyer in the Double Auction.

  Attributes:
    name: Unique identifier.
    redemption_values: Sorted list (descending) of redemption values (private to this buyer).
    tokens_bought: Number of tokens purchased so far this period.
    profit: Accumulated profit from trades.
  """

  def __init__(self, name: str, redemption_values: List[int]) -> None:
    """Inits a Buyer with name and redemption values.

    Args:
      name: Unique name.
      redemption_values: List of private values for each token (e.g. [320, 300]).
    """
    self.name = name
    self.redemption_values = sorted(redemption_values, reverse=True)
    self.tokens_bought = 0
    self.profit = 0

  def can_buy(self) -> bool:
    """Indicates if buyer can still buy tokens."""
    return self.tokens_bought < len(self.redemption_values)

  def next_value(self) -> Optional[int]:
    """Returns redemption value for next token to be purchased, if any."""
    if self.can_buy():
      return self.redemption_values[self.tokens_bought]
    return None

  def place_bid(self,
                current_bid: Optional[int],
                min_price: int,
                max_price: int) -> Optional[int]:
    """Determines a potential bid price given the current best bid.

    Strategy:
      - Buyer picks a random bid in [max(current_bid+1, min_price) ... next_value()]
      - Must not exceed max_price or next_value().

    Args:
      current_bid: The highest existing bid, or None if none.
      min_price: Minimum valid price for a bid.
      max_price: Maximum valid price for a bid.

    Returns:
      A new integer bid, or None if none is possible.
    """
    if not self.can_buy():
      return None
    lb = (current_bid + 1) if current_bid is not None else min_price
    ub = min(self.next_value(), max_price)
    if lb > ub:
      return None
    return random.randint(lb, ub)

  def accept_offer(self, current_offer: Optional[int]) -> bool:
    """Decides whether to accept the current offer price.

    Strategy:
      - Accept if next_value() >= current_offer.

    Args:
      current_offer: The lowest existing offer, or None if none.

    Returns:
      True if accepting the offer, False otherwise.
    """
    if (not self.can_buy()) or (current_offer is None):
      return False
    return (self.next_value() - current_offer) >= 0

  def record_purchase(self, price: int) -> None:
    """Updates buyer's profit and token count after buying one token.

    Args:
      price: Transaction price paid by buyer.
    """
    if not self.can_buy():
      return
    val = self.next_value()
    self.profit += (val - price)
    self.tokens_bought += 1


class Seller:
  """Represents a seller in the Double Auction.

  Attributes:
    name: Unique identifier.
    cost_values: Sorted list (ascending) of costs (private to this seller).
    tokens_sold: Number of tokens sold so far this period.
    profit: Accumulated profit from trades.
  """

  def __init__(self, name: str, cost_values: List[int]) -> None:
    """Inits a Seller with name and cost values.

    Args:
      name: Unique name.
      cost_values: List of private cost values for each token (e.g. [250, 270]).
    """
    self.name = name
    self.cost_values = sorted(cost_values)
    self.tokens_sold = 0
    self.profit = 0

  def can_sell(self) -> bool:
    """Indicates if seller can still sell tokens."""
    return self.tokens_sold < len(self.cost_values)

  def next_cost(self) -> Optional[int]:
    """Returns cost for next token to be sold, if any."""
    if self.can_sell():
      return self.cost_values[self.tokens_sold]
    return None

  def place_offer(self,
                  current_offer: Optional[int],
                  min_price: int,
                  max_price: int) -> Optional[int]:
    """Determines a potential offer price given the current best (lowest) offer.

    Strategy:
      - Seller picks a random offer in [max(min_price, next_cost()) ... current_offer - 1].
      - Must not exceed max_price or go below min_price.

    Args:
      current_offer: The lowest existing offer, or None if none.
      min_price: Minimum valid price for an offer.
      max_price: Maximum valid price for an offer.

    Returns:
      A new integer offer, or None if none is possible.
    """
    if not self.can_sell():
      return None
    upper_bound = (current_offer - 1) if current_offer is not None else max_price
    lower_bound = max(min_price, self.next_cost())
    if lower_bound > upper_bound:
      return None
    return random.randint(lower_bound, upper_bound)

  def accept_bid(self, current_bid: Optional[int]) -> bool:
    """Decides whether to accept the current bid price.

    Strategy:
      - Accept if current_bid >= next_cost().

    Args:
      current_bid: The highest existing bid, or None if none.

    Returns:
      True if accepting the bid, False otherwise.
    """
    if (not self.can_sell()) or (current_bid is None):
      return False
    return (current_bid - self.next_cost()) >= 0

  def record_sale(self, price: int) -> None:
    """Updates seller's profit and token count after selling one token.

    Args:
      price: Transaction price received by seller.
    """
    if not self.can_sell():
      return
    cst = self.next_cost()
    self.profit += (price - cst)
    self.tokens_sold += 1


class Auction:
  """Manages a Double Auction simulation with multiple buyers/sellers and discrete steps.

  Attributes:
    buyers: List of Buyer instances.
    sellers: List of Seller instances.
    min_price: Minimum allowed price.
    max_price: Maximum allowed price.
    max_time_steps: Number of time steps per period.
    current_bid: Tuple (bid_price, Buyer) or None.
    current_offer: Tuple (offer_price, Seller) or None.
    df_steps: List of dict logs for each step (bid-offer or buy-sell).
    df_trades: List of dict logs for each executed trade.
  """

  def __init__(self,
               buyers: List[Buyer],
               sellers: List[Seller],
               min_price: int = 1,
               max_price: int = 1000,
               max_time_steps: int = 5) -> None:
    """Inits Auction with given participants and parameters.

    Args:
      buyers: List of Buyer objects.
      sellers: List of Seller objects.
      min_price: Lowest valid integer price.
      max_price: Highest valid integer price.
      max_time_steps: Time steps per period.
    """
    self.buyers = buyers
    self.sellers = sellers
    self.min_price = min_price
    self.max_price = max_price
    self.max_time_steps = max_time_steps
    self.current_bid: Optional[Tuple[int, Buyer]] = None
    self.current_offer: Optional[Tuple[int, Seller]] = None
    self.df_steps = []
    self.df_trades = []

  def reset_period(self) -> None:
    """Resets token counters and current quotes for a new period."""
    for b in self.buyers:
      b.tokens_bought = 0
    for s in self.sellers:
      s.tokens_sold = 0
    self.current_bid = None
    self.current_offer = None

  def run_period(self, round_id: int = 1, period_id: int = 1) -> None:
    """Executes one full period with a fixed number of time steps.

    Args:
      round_id: For logging, which round number.
      period_id: For logging, which period number in the round.
    """
    for t in range(1, self.max_time_steps + 1):
      self._collect_bids_and_offers(round_id, period_id, t)
      self._attempt_trade(round_id, period_id, t)

  def _collect_bids_and_offers(self, r: int, p: int, t: int) -> None:
    """Collects new bids and offers from participants, updates current best bid/offer."""
    bids = []
    for b in self.buyers:
      proposed_bid = b.place_bid(
          self.current_bid[0] if self.current_bid else None,
          self.min_price,
          self.max_price)
      if proposed_bid is not None:
        bids.append((proposed_bid, b))
    if bids:
      highest_price = max(x[0] for x in bids)
      tied_bidders = [x for x in bids if x[0] == highest_price]
      self.current_bid = random.choice(tied_bidders)

    offers = []
    for s in self.sellers:
      proposed_offer = s.place_offer(
          self.current_offer[0] if self.current_offer else None,
          self.min_price,
          self.max_price)
      if proposed_offer is not None:
        offers.append((proposed_offer, s))
    if offers:
      lowest_price = min(x[0] for x in offers)
      tied_sellers = [x for x in offers if x[0] == lowest_price]
      self.current_offer = random.choice(tied_sellers)

    self.df_steps.append({
        'round': r,
        'period': p,
        'time': t,
        'new_bid': self.current_bid[0] if self.current_bid else None,
        'bidder': self.current_bid[1].name if self.current_bid else None,
        'new_offer': self.current_offer[0] if self.current_offer else None,
        'offerer': self.current_offer[1].name if self.current_offer else None,
        'type': 'bid-offer'
    })

  def _attempt_trade(self, r: int, p: int, t: int) -> None:
    """Checks if the current bidder and offerer accept each other's price and executes a trade."""
    if not self.current_bid or not self.current_offer:
      return
    bid_price, buyer = self.current_bid
    offer_price, seller = self.current_offer
    buyer_accepts = buyer.accept_offer(offer_price)
    seller_accepts = seller.accept_bid(bid_price)
    trade_occurred = False
    price_used: Optional[int] = None
    if buyer_accepts and seller_accepts:
      price_used = random.choice([bid_price, offer_price])
      buyer.record_purchase(price_used)
      seller.record_sale(price_used)
      trade_occurred = True
      self.current_bid = None
      self.current_offer = None
    elif buyer_accepts and not seller_accepts:
      price_used = offer_price
      buyer.record_purchase(price_used)
      seller.record_sale(price_used)
      trade_occurred = True
      self.current_bid = None
      self.current_offer = None
    elif seller_accepts and not buyer_accepts:
      price_used = bid_price
      buyer.record_purchase(price_used)
      seller.record_sale(price_used)
      trade_occurred = True
      self.current_bid = None
      self.current_offer = None

    if trade_occurred:
      self.df_trades.append({
          'round': r,
          'period': p,
          'time': t,
          'buyer': buyer.name,
          'seller': seller.name,
          'price': price_used
      })

    self.df_steps.append({
        'round': r,
        'period': p,
        'time': t,
        'new_bid': bid_price,
        'bidder': buyer.name,
        'new_offer': offer_price,
        'offerer': seller.name,
        'type': 'buy-sell',
        'trade': trade_occurred,
        'price': price_used
    })

  def summarize(self) -> None:
    """Prints a simple summary of results after the period or game ends."""
    print("=== Auction Summary ===")
    for b in self.buyers:
      print(f"Buyer {b.name} => tokens_bought={b.tokens_bought}, profit={b.profit}")
    for s in self.sellers:
      print(f"Seller {s.name} => tokens_sold={s.tokens_sold}, profit={s.profit}")
    print("=======================")

  def get_history(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Returns DataFrames of all steps and trades for external analysis."""
    return (pd.DataFrame(self.df_steps), pd.DataFrame(self.df_trades))


def main() -> None:
  """Example usage of the Auction class."""
  random.seed(42)
  b1 = Buyer(name="B1", redemption_values=[320, 300])
  b2 = Buyer(name="B2", redemption_values=[310, 290])
  s1 = Seller(name="S1", cost_values=[250, 270])
  s2 = Seller(name="S2", cost_values=[260, 280])
  auction = Auction([b1, b2], [s1, s2], min_price=1, max_price=400, max_time_steps=5)
  auction.run_period(round_id=1, period_id=1)
  auction.summarize()
  steps_df, trades_df = auction.get_history()
  print("\nSteps DataFrame:\n", steps_df)
  print("\nTrades DataFrame:\n", trades_df)


if __name__ == "__main__":
  main()


=== Auction Summary ===
Buyer B1 => tokens_bought=2, profit=9
Buyer B2 => tokens_bought=2, profit=50
Seller S1 => tokens_sold=2, profit=62
Seller S2 => tokens_sold=2, profit=39

Steps DataFrame:
    round  period  time  new_bid bidder  new_offer offerer       type  trade  \
0      1       1     1       58     B1        312      S1  bid-offer    NaN   
1      1       1     1       58     B1        312      S1   buy-sell   True   
2      1       1     2      280     B2        268      S2  bid-offer    NaN   
3      1       1     2      280     B2        268      S2   buy-sell   True   
4      1       1     3      120     B2        320      S1  bid-offer    NaN   
5      1       1     3      120     B2        320      S1   buy-sell  False   
6      1       1     4      235     B2        270      S1  bid-offer    NaN   
7      1       1     4      235     B2        270      S1   buy-sell   True   
8      1       1     5      175     B1        299      S2  bid-offer    NaN   
9      1      